# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO

For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
# import datetime
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

import datetime
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now
now()

datetime.datetime(2022, 4, 2, 9, 10, 6, 410038)

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 20

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded
# channels_last
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1, # shows the steps in every epoch
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [11]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/20
230/230 [==============================] - 2s 11ms/step - loss: 1.6089 - accuracy: 0.1938 - val_loss: 1.5985 - val_accuracy: 0.2010
Epoch 2/20
230/230 [==============================] - 2s 10ms/step - loss: 1.5946 - accuracy: 0.2262 - val_loss: 1.5827 - val_accuracy: 0.2080
Epoch 3/20
230/230 [==============================] - 2s 10ms/step - loss: 1.5789 - accuracy: 0.2632 - val_loss: 1.5660 - val_accuracy: 0.2386
Epoch 4/20
230/230 [==============================] - 2s 10ms/step - loss: 1.5641 - accuracy: 0.3015 - val_loss: 1.5478 - val_accuracy: 0.3150
Epoch 5/20
230/230 [==============================] - 2s 10ms/step - loss: 1.5455 - accuracy: 0.3451 - val_loss: 1.5271 - val_accuracy: 0.3993
Epoch 6/20
230/230 [==============================] - 2s 10ms/step - loss: 1.5247 - accuracy: 0.3966 - val_loss: 1.5030 - val_accuracy: 0.4804
Epoch 7/20
230/230 [==============================] - 2s 10ms/step - l

### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [16]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/20
240/240 [==============================] - 2s 7ms/step - loss: 1.6029 - accuracy: 0.3788 - val_loss: 1.4769 - val_accuracy: 0.4275
Epoch 2/20
240/240 [==============================] - 1s 6ms/step - loss: 1.4718 - accuracy: 0.4231 - val_loss: 1.3524 - val_accuracy: 0.4721
Epoch 3/20
240/240 [==============================] - 1s 5ms/step - loss: 1.3625 - accuracy: 0.4641 - val_loss: 1.2444 - val_accuracy: 0.5359
Epoch 4/20
240/240 [==============================] - 1s 6ms/step - loss: 1.2669 - accuracy: 0.5109 - val_loss: 1.1503 - val_accuracy: 0.5986
Epoch 5/20
240/240 [==============================] - 1s 6ms/step - loss: 1.1879 - accuracy: 0.5461 - val_loss: 1.0678 - val_accuracy: 0.6632
Epoch 6/20
240/240 [==============================] - 1s 6ms/step - loss: 1.1088 - accuracy: 0.5984 - val_loss: 0.9925 - val_accuracy: 0.7194
Epoch 7/20
240/240 [==============================] - 1s 6ms/step - loss: 1.

Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.

In [17]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [18]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/20
240/240 [==============================] - 2s 10ms/step - loss: 1.5892 - accuracy: 0.2750 - val_loss: 1.5595 - val_accuracy: 0.4758
Epoch 2/20
240/240 [==============================] - 2s 10ms/step - loss: 1.5510 - accuracy: 0.3728 - val_loss: 1.5182 - val_accuracy: 0.6678
Epoch 3/20
240/240 [==============================] - 2s 10ms/step - loss: 1.5101 - accuracy: 0.4636 - val_loss: 1.4730 - val_accuracy: 0.7531
Epoch 4/20
240/240 [==============================] - 2s 10ms/step - loss: 1.4668 - accuracy: 0.5408 - val_loss: 1.4210 - val_accuracy: 0.7893
Epoch 5/20
240/240 [==============================] - 2s 10ms/step - loss: 1.4142 - accuracy: 0.6024 - val_loss: 1.3589 - val_accuracy: 0.8144
Epoch 6/20
240/240 [==============================] - 2s 10ms/step - loss: 1.3550 - accuracy: 0.6544 - val_loss: 1.2865 - val_accuracy: 0.8465
Epoch 7/20
240/240 [==============================] - 2s 10ms/step - l

In [19]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [20]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [21]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/20
230/230 [==============================] - 1s 6ms/step - loss: 1.9811 - accuracy: 0.3141 - val_loss: 1.7784 - val_accuracy: 0.3567
Epoch 2/20
230/230 [==============================] - 1s 6ms/step - loss: 1.7929 - accuracy: 0.3320 - val_loss: 1.6030 - val_accuracy: 0.3925
Epoch 3/20
230/230 [==============================] - 1s 5ms/step - loss: 1.6327 - accuracy: 0.3698 - val_loss: 1.4492 - val_accuracy: 0.4351
Epoch 4/20
230/230 [==============================] - 1s 5ms/step - loss: 1.5009 - accuracy: 0.4087 - val_loss: 1.3155 - val_accuracy: 0.4911
Epoch 5/20
230/230 [==============================] - 1s 5ms/step - loss: 1.3906 - accuracy: 0.4502 - val_loss: 1.2016 - val_accuracy: 0.5538
Epoch 6/20
230/230 [==============================] - 1s 5ms/step - loss: 1.2847 - accuracy: 0.4959 - val_loss: 1.1060 - val_accuracy: 0.6091
Epoch 7/20
230/230 [==============================] - 1s 6ms/step - loss: 1.

---
### Machine Learning Foundation (C) 2020 IBM Corporation